In [2]:
import os
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
import time
import csv


TXTS_PATH = "transcripts"
OUT_PATH = "summaries_russian"
os.makedirs(OUT_PATH, exist_ok=True)

CHUNK_TOKENS = 1024
SUMMARY_TOKENS = 150

DEVICE = 0 if torch.cuda.is_available() else -1
LANG_CODE = "ru_RU"

def chunk_text(text, tokenizer, max_tokens):
    import nltk
    nltk.download('punkt', quiet=True)
    from nltk.tokenize import sent_tokenize
    sentences = sent_tokenize(text, language='russian')
    chunks, current = [], ""
    for sent in sentences:
        if len(tokenizer.encode(current + sent)) < max_tokens:
            current += " " + sent
        else:
            if current: chunks.append(current)
            current = sent
    if current:
        chunks.append(current)
    return chunks

def summarize_with_mbart50(text, model, tokenizer):
    chunks = chunk_text(text, tokenizer, CHUNK_TOKENS)
    summaries = []
    for chunk in chunks:
        inputs = tokenizer(
            chunk, return_tensors="pt", max_length=CHUNK_TOKENS, truncation=True
        ).to(model.device)
        with torch.no_grad():
            summary_ids = model.generate(
                **inputs,
                max_length=SUMMARY_TOKENS, 
                num_beams=4, 
                forced_bos_token_id=tokenizer.lang_code_to_id[LANG_CODE]
            )
        out = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(out)
    merged_summary = " ".join(summaries)
    return merged_summary

def summarize_with_rut5(text, pipe):
    chunks = chunk_text(text, pipe.tokenizer, CHUNK_TOKENS)
    # ruT5 is T5, so use native summarization pipeline
    summaries = pipe(chunks, max_length=SUMMARY_TOKENS, min_length=20, truncation=True, batch_size=4)
    merged = " ".join([out['summary_text'] for out in summaries])
    return merged

# LOAD MODELS ONCE
mbart_tokenizer = MBart50TokenizerFast.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')
mbart_tokenizer.src_lang = LANG_CODE
mbart_model = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-many-to-many-mmt').to(DEVICE)

ruT5_model_name = "ai-forever/ruT5-base"
ruT5_pipe = pipeline("summarization", model=ruT5_model_name, tokenizer=ruT5_model_name, device=DEVICE)

timing_report = []  # List of dicts: one row per run

for fname in os.listdir(TXTS_PATH):
    if not fname.endswith(".txt"): continue
    fpath = os.path.join(TXTS_PATH, fname)
    with open(fpath, "r", encoding="utf-8") as f:
        text = f.read()
    
    # MBART-50
    outname_mbart = os.path.join(OUT_PATH, f"{fname}.mbart.summary.txt")
    if not os.path.exists(outname_mbart):
        t0 = time.time()
        summary = summarize_with_mbart50(text, mbart_model, mbart_tokenizer)
        elapsed = time.time() - t0
        with open(outname_mbart, "w", encoding="utf-8") as fout:
            fout.write(summary)
        print(f"  Wrote {outname_mbart}, time: {elapsed:.1f}s")
        timing_report.append({'filename': fname, 'model': 'mbart50', 'seconds': elapsed})
        
    # ruT5
    outname_rut5 = os.path.join(OUT_PATH, f"{fname}.rut5.summary.txt")
    if not os.path.exists(outname_rut5):
        t0 = time.time()
        summary = summarize_with_rut5(text, ruT5_pipe)
        elapsed = time.time() - t0
        with open(outname_rut5, "w", encoding="utf-8") as fout:
            fout.write(summary)
        print(f"  Wrote {outname_rut5}, time: {elapsed:.1f}s")
        timing_report.append({'filename': fname, 'model': 'ruT5', 'seconds': elapsed})

    # TextRank
    try:
        from summa.summarizer import summarize as textrank_summarize
        outname_textrank = os.path.join(OUT_PATH, f"{fname}.textrank.summary.txt")
        if not os.path.exists(outname_textrank):
            t0 = time.time()
            summary = textrank_summarize(text, language="russian", ratio=0.07)
            elapsed = time.time() - t0
            with open(outname_textrank, "w", encoding="utf-8") as fout:
                fout.write(summary)
            print(f"  Wrote {outname_textrank}, time: {elapsed:.1f}s")
            timing_report.append({'filename': fname, 'model': 'textrank', 'seconds': elapsed})
    except ImportError:
        print("summa not installed, skipping textrank.")


print("DONE.")


  Wrote summaries_russian/recognized_small_20m20s.txt.mbart.summary.txt, time: 105.8s


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  Wrote summaries_russian/recognized_small_20m20s.txt.rut5.summary.txt, time: 33.6s
summa not installed, skipping textrank.
  Wrote summaries_russian/recognized_medium_24m15s.txt.mbart.summary.txt, time: 77.4s
  Wrote summaries_russian/recognized_medium_24m15s.txt.rut5.summary.txt, time: 22.0s
summa not installed, skipping textrank.
  Wrote summaries_russian/recognized_large_33m5s.txt.mbart.summary.txt, time: 81.9s
  Wrote summaries_russian/recognized_large_33m5s.txt.rut5.summary.txt, time: 31.8s
summa not installed, skipping textrank.
  Wrote summaries_russian/recognized_tiny_14m19s.txt.mbart.summary.txt, time: 174.1s
  Wrote summaries_russian/recognized_tiny_14m19s.txt.rut5.summary.txt, time: 74.8s
summa not installed, skipping textrank.
DONE.


In [3]:
# Save timing results for analysis
with open(os.path.join(OUT_PATH, 'timing_report.csv'), "w", encoding="utf-8", newline='') as f_csv:
    writer = csv.DictWriter(f_csv, fieldnames=['filename', 'model', 'seconds'])
    writer.writeheader()
    writer.writerows(timing_report)
print("Timing CSV report saved:", os.path.join(OUT_PATH, 'timing_report.csv'))

Timing CSV report saved: summaries_russian/timing_report.csv


# LLM chats

In [13]:
!pip install transformers accelerate torch sentencepiece tqdm --extra-index-url https://pypi.org/simple/

Looking in indexes: https://pypi.yandex-team.ru/simple/, https://pypi.org/simple/


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import time

os.environ["HF_TOKEN"] = "removed"

TRANSCRIPTS_PATH = "transcripts"
OUTPUT_PATH = "summaries_russian_llm"
os.makedirs(OUTPUT_PATH, exist_ok=True)

MODELS = {
    "mistral": "mistralai/Mistral-7B-Instruct-v0.2",
    "qwen": "Qwen/Qwen1.5-7B-Chat",
    "llama": "meta-llama/Llama-2-7b-chat-hf",
}

PROMPT_TEMPLATES = {
    "mistral": "Сделай краткое и информативное резюме следующего текста собрания на русском языке, выдели принятые решения и action-items.:\n\n{}",
    "llama": "Сделай краткое и информативное резюме следующего текста собрания на русском языке, выдели принятые решения и action-items.:\n\n{}",
    "qwen": "Сделай краткое и информативное резюме следующего текста собрания на русском языке, выдели принятые решения и action-items.:\n\n{}",
}

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CHUNK_TOKENS = 2048
GENERATE_TOKENS = 512

def chunk_text(text, tokenizer, max_tokens):
    """Splits Russian text into token-length chunks at sentence boundaries."""
    import nltk
    nltk.download('punkt', quiet=True)
    from nltk.tokenize import sent_tokenize

    sentences = sent_tokenize(text, language='russian')
    chunks = []
    current_chunk = ""
    for sent in sentences:
        if len(tokenizer.encode(current_chunk + " " + sent)) < max_tokens:
            current_chunk += " " + sent
        else:
            if current_chunk: chunks.append(current_chunk.strip())
            current_chunk = sent
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def generate_summary(
    model, tokenizer, prompt, device, chunk_tokens=2048, gen_tokens=512
):
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(
        prompt,
        return_tensors='pt',
        truncation=True,
        max_length=chunk_tokens,
        padding="max_length"
    )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # print("Model device:", next(model.parameters()).device)
    # print("input_ids:", input_ids.device)
    # print("attention_mask:", attention_mask.device)
    
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=chunk_tokens+gen_tokens,
            temperature=0.3,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    result = tokenizer.decode(
        output_ids[0][input_ids.shape[-1]:],
        skip_special_tokens=True
    )
    return result.strip()


timing_report = []
for model_key, model_name in MODELS.items():
    print(f"> Loading {model_key} ({model_name}) ...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map=None,
        trust_remote_code=True,        # for Qwen especially
    )
    model = model.to(DEVICE)
    model.eval()
    prompt_template = PROMPT_TEMPLATES[model_key]
    for fname in os.listdir(TRANSCRIPTS_PATH):
        if not fname.endswith('.txt'):
            continue
        transcript_path = os.path.join(TRANSCRIPTS_PATH, fname)
        outname = os.path.join(OUTPUT_PATH, f"{fname}.{model_key}.summary.txt")
        if os.path.exists(outname):
            print(f"Skip {outname}, exists.")
            continue
        with open(transcript_path, "r", encoding="utf-8") as fin:
            text = fin.read()
        chunks = chunk_text(text, tokenizer, CHUNK_TOKENS)
        summaries = []
        t0 = time.time()
        for chunk in tqdm(chunks, desc=f"{fname} ({model_key})"):
            prompt = prompt_template.format(chunk)
            summary = generate_summary(model, tokenizer, prompt, DEVICE)
            summaries.append(summary)
        final_summary = "\n".join(summaries)
        elapsed = time.time() - t0
        with open(outname, "w", encoding="utf-8") as fout:
            fout.write(final_summary)
        print(f">>> {fname} ({model_key}) done in {elapsed:.1f} sec ({len(chunks)} chunks)")
        timing_report.append({'filename': fname, 'model': model_key, 'seconds': elapsed})

import csv
with open(os.path.join(OUTPUT_PATH, 'timing_report.csv'), "w", encoding="utf-8", newline='') as f_csv:
    writer = csv.DictWriter(f_csv, fieldnames=['filename', 'model', 'seconds'])
    writer.writeheader()
    writer.writerows(timing_report)
print("Timing report written:", os.path.join(OUTPUT_PATH, 'timing_report.csv'))


> Loading mistral (mistralai/Mistral-7B-Instruct-v0.2) ...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

recognized_small_20m20s.txt (mistral):   0%|                                                                                    | 0/27 [00:00<?, ?it/s]

Model device: cuda:0
input_ids: cuda:0
attention_mask: cuda:0


recognized_small_20m20s.txt (mistral):   4%|██▊                                                                         | 1/27 [00:35<15:21, 35.45s/it]

...

recognized_tiny_14m19s.txt (qwen):  97%|████████████████████████████████████████████████████████████████████████████▋  | 34/35 [06:26<00:05,  5.97s/it]

Model device: cuda:0
input_ids: cuda:0
attention_mask: cuda:0


recognized_tiny_14m19s.txt (qwen): 100%|███████████████████████████████████████████████████████████████████████████████| 35/35 [06:26<00:00, 11.05s/it]


>>> recognized_tiny_14m19s.txt (qwen) done in 386.9 sec (35 chunks)
> Loading llama (meta-llama/Llama-2-7b-chat-hf) ...


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-6816bc29-54285b27730255e442d2a8b7;61ed4734-8984-403f-b44c-5ef614a86b2b)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-chat-hf to ask for access.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

with open("transcripts/recognized_large_33m5s.txt", "r", encoding="utf-8") as f:
    text = f.read()


text = text[:5000]

# prepare the model input
prompt = "Сделай краткое и информативное резюме следующего текста собрания на русском языке, выдели принятые решения и action-items.:\n\n"
messages = [
    {"role": "user", "content": prompt + text}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048 #32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

thinking content: <think>
Хорошо, мне нужно сделать краткое и информативное резюме собрания на русском языке, выделив принятые решения и action-items. Начну с того, что внимательно прочитаю текст, чтобы понять основные моменты.

Сначала текст говорит о важности искусственного интеллекта (ИИ) как ключевого направления для будущего продуктов. Важно отметить, что некоторые коллеги поднимают вопросы, связанные с внедрением ИИ, и нужно совместно их решать. Также упоминается возможное создание нового управления для ускорения внедрения ИИ. 

Далее, говорится о том, что ИИ может ускорить развитие, как это происходило в истории человечества через появление письменности. Примеры: чтение большого количества текстов и обучение на них. ИИ способен обрабатывать огромные объемы данных быстрее человека, что может привести к трансформации подходов к жизни и бизнесу. 

Важно подчеркнуть, что компания должна активно внедрять ИИ, чтобы не отставать от конкурентов. Также упоминается, что нужно сформировать

In [5]:
import torch
torch.cuda.empty_cache()


In [2]:
import os
import nltk
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from tqdm import tqdm

nltk.download('punkt', quiet=True)
from nltk.tokenize import sent_tokenize

model_name = "Qwen/Qwen3-8B"       # << DO NOT CHANGE THIS
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

# INPUT_PATH = "transcripts/recognized_large_33m5s.txt"

INPUT_PATH = "summaries_russian_llm/qwen3_summary_summary.txt"
OUTPUT_PATH = "summaries_russian_llm/qwen3_summary_summary_global.txt"
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

PROMPT_CHUNK = (
    "Сделай краткое и информативное резюме следующего текста собрания на русском языке, "
    "выдели принятые решения и action-items:\n\n"
)

PROMPT_GLOBAL = (
    "Вот несколько кратких резюме разных частей собрания. На их основе создай обобщённое, "
    "сжатое итоговое резюме всего собрания, выдели только основные решения и важнейшие action items:\n\n"
)

def chunk_text(text, tokenizer, max_tokens=2000):
    sentences = sent_tokenize(text, language='russian')
    chunks = []
    current = ""
    for sent in sentences:
        test = (current + " " + sent).strip()
        token_count = len(tokenizer.encode(test))
        if token_count < max_tokens:
            current = test
        else:
            if current:
                chunks.append(current)
            current = sent
    if current:
        chunks.append(current)
    return chunks

with open(INPUT_PATH, "r", encoding="utf-8") as f:
    text = f.read()

chunks = chunk_text(text, tokenizer, max_tokens=2000)
print(f"Chunked transcript into {len(chunks)} segments.")

summaries = []
for idx, chunk in enumerate(tqdm(chunks, desc="Summarizing chunks")):
    messages = [{"role": "user", "content": PROMPT_GLOBAL + chunk}]
    chat_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        enable_thinking=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([chat_text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        out_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    summary_chunk = tokenizer.decode(
        out_ids[0][model_inputs.input_ids.shape[-1]:], skip_special_tokens=True
    ).strip()
    summaries.append(summary_chunk)

# ---- Second pass: summarize the summaries ----
global_summary_prompt = PROMPT_GLOBAL + "\n\n".join(summaries)
# messages = [{"role": "user", "content": global_summary_prompt}]
# chat_text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# model_inputs = tokenizer([chat_text], return_tensors="pt").to(model.device)
# with torch.no_grad():
#     out_ids = model.generate(
#         **model_inputs,
#         max_new_tokens=700,
#         temperature=0.1,
#         do_sample=False,
#         pad_token_id=tokenizer.eos_token_id,
#         eos_token_id=tokenizer.eos_token_id
#     )
# final_summary = tokenizer.decode(
#     out_ids[0][model_inputs.input_ids.shape[-1]:], skip_special_tokens=True
# ).strip()

final_summary = global_summary_prompt

with open(OUTPUT_PATH, "w", encoding="utf-8") as fout:
    fout.write(final_summary)

print(f"\nFinal summary written to {OUTPUT_PATH}\n")


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Chunked transcript into 3 segments.


/home/mkudriashov/tts/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/mkudriashov/tts/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/mkudriashov/tts/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Summarizing chunks: 100%|███████████████████████████


Final summary written to summaries_russian_llm/qwen3_summary_summary_global.txt

